# Validation Set 1.2: diffuPy + PathMe  

In [1]:
import os
import itertools

import time
import copy

dir_path = os.path.dirname(os.path.realpath('__file__'))

In [2]:
from openpyxl import load_workbook
from collections import defaultdict
import networkx as nx

In [27]:
import pybel
import pybel_tools as pbt

from pybel import BELGraph
from pybel.constants import RELATION, ANNOTATIONS
from pybel.dsl import Abundance, BiologicalProcess, CentralDogma, ListAbundance, Reaction
from pybel.struct import get_subgraph_by_annotation_value
from pybel.struct.summary import count_functions

pybel.get_version()

'0.13.2'

In [4]:
from pathme.constants import REACTOME_BEL, KEGG_BEL, WIKIPATHWAYS_BEL, PATHME_DIR
from pathme.export_utils import to_gml

In [5]:
from diffuPy.diffuse import diffuse
from diffuPy.kernels import regularised_laplacian_kernel
from diffuPy.matrix import Matrix, LaplacianMatrix
from diffuPy.input_mapping import generate_categoric_input_from_labels, get_mapping, get_mapping_subsets, get_mapping_two_dim_subsets
from diffuPy.validation_sets_utils import parse_set1
from diffuPy.utils import print_dict_dimentions, get_labels_set_from_dict, reduce_dict_dimention
from diffuPy.views import show_heatmap

from diffuPy.miscellaneous import get_label_list_graph


In [6]:
# TODO
import nbimporter
from global_database_comparison import get_set_database, calculate_database_sets_as_dict

Importing Jupyter notebook from global_database_comparison.ipynb


## 1. Load Data Set 1: Input Scores

In [7]:
dataset1_labels_by_omics = parse_set1(os.path.join(dir_path, 'validation', 'set1.xlsx'))

In [8]:
print_dict_dimentions(dataset1_labels_by_omics, 'Dataset1 imported labels:')

Dataset1 imported labels:
Total number of genes: 4941  
Total number of micrornas: 100  
Total number of metabolite: 21  
Total: 5062 


In [9]:
dataset1_all_labels = get_labels_set_from_dict(dataset1_labels_by_omics)

## 2. Load Backrgound Graph Universe

### 2.1. PathMeUniverse import

##### Database argument in node data

In [10]:
pathme_universe_graph_no_flatten = pybel.from_pickle(os.path.join(PATHME_DIR, 'universe', 'pathme_universe_bel_graph_no_flatten.bel.pickle'))


In [11]:
pathme_universe_graph_no_flatten.summarize()

PathMe Universe v1.0.0
Number of Nodes: 49697
Number of Edges: 104728
Number of Citations: 7706
Number of Authors: 0
Network Density: 4.24E-05
Number of Components: 7018
Number of Warnings: 0


##### Check isolates / Unconnected nodes

In [12]:
nx.number_of_isolates(pathme_universe_graph_no_flatten)

6534

In [13]:
pathme_universe_graph_no_flatten.remove_nodes_from({
    node
    for node in nx.isolates(pathme_universe_graph_no_flatten)
})

In [14]:
nx.number_of_isolates(pathme_universe_graph_no_flatten)

0

##### Export to gml

In [15]:
to_gml(pathme_universe_graph_no_flatten, os.path.join(PATHME_DIR, 'pathme_universe_graph_no_flatten.gml'))

In [16]:
nx_pathme_universe_graph_no_flatten = nx.read_gml(os.path.join(PATHME_DIR, 'pathme_universe_graph_no_flatten.gml'))

In [17]:
nx.info(nx_pathme_universe_graph_no_flatten)

'Name: \nType: MultiDiGraph\nNumber of nodes: 43105\nNumber of edges: 104728\nAverage in degree:   2.4296\nAverage out degree:   2.4296'

In [18]:
nx.number_of_isolates(nx_pathme_universe_graph_no_flatten)

0

### 2.2. Get labels subsets by entity type (omic) and database

##### PathMe package subsets

In [19]:
def get_labels_by_db_and_omic_from_pathme(databases):
    db_entites = defaultdict(dict)
    entites_db = defaultdict(lambda: defaultdict(set))
    
    for db in databases:
        genes, mirna, metabolites, bps = get_set_database(db)
        db_entites[db] = {'genes': genes, 'mirna': mirna, 'metabolites': metabolites, 'bps': bps}
        
        for entity_type, entities in db_entites[db].items():
            entites_db[entity_type][db] = entities
            
    return db_entites, entites_db

In [20]:
bg_labels_from_pathme_by_db_and_omic, bg_labels_from_pathme_by_omic_and_db = get_labels_by_db_and_omic_from_pathme(['reactome', 'kegg', 'wikipathways'])

print_dict_dimentions(bg_labels_from_pathme_by_db_and_omic, 'Entities in PathMe package :')



Entities in PathMe package :
Total number of reactome: genes(8653), mirna(13), metabolites(2712), bps(2219),  
Total number of kegg: genes(7656), mirna(149), metabolites(4421), bps(418),  
Total number of wikipathways: genes(3361), mirna(91), metabolites(657), bps(135),  
Total: 30485 


In [21]:
bg_labels_from_pathme_by_omic = reduce_dict_dimention(bg_labels_from_pathme_by_omic_and_db)
mirnas = bg_labels_from_pathme_by_omic['mirna']

#####  PathMeUniverse Graph Subsets

In [22]:
def get_labels_by_db_and_omic_from_graph(graph):
    
    db_subsets_pmu = defaultdict(set)
    db_entites_pmu = defaultdict(dict)

    #entity_type_map = {'Gene':'genes', 'mirna_nodes':'mirna', 'Abundance':'metabolites', 'BiologicalProcess':'bps'}

    for u, v, k in graph.edges(keys=True):
        if ANNOTATIONS not in graph[u][v][k]:
            continue
            
        if 'database' not in graph[u][v][k][ANNOTATIONS]:
            continue
            
        for database in graph[u][v][k][ANNOTATIONS]['database']:
            db_subsets_pmu[database].add(u)
            db_subsets_pmu[database].add(v)
        
    for database, nodes in db_subsets_pmu.items():
        db_entites_pmu[database] = calculate_database_sets_as_dict(nodes, database)
        
    return db_entites_pmu, db_subsets_pmu

In [23]:
bg_labels_from_pathmeuniverse_by_db_and_omic, bg_labels_from_pathmeuniverse_by_omic_and_db =  get_labels_by_db_and_omic_from_graph(pathme_universe_graph_no_flatten)

print_dict_dimentions(bg_labels_from_pathmeuniverse_by_db_and_omic, 'Entities in PathMeUniverse :')


Entities in PathMeUniverse :
Total number of kegg: gene_nodes(6037), mirna_nodes(122), metabolite_nodes(1753), bp_nodes(112),  
Total number of reactome: gene_nodes(8634), mirna_nodes(13), metabolite_nodes(2681), bp_nodes(0),  
Total number of wikipathways: gene_nodes(2555), mirna_nodes(91), metabolite_nodes(634), bp_nodes(135),  
Total: 22767 


### 2.3. Get subgraphs subsets by database

In [24]:
kegg_pathme_graph_no_flatten = get_subgraph_by_annotation_value(pathme_universe_graph_no_flatten, 'database', 'kegg')
reactome_pathme_universe_graph_no_flatten = get_subgraph_by_annotation_value(pathme_universe_graph_no_flatten, 'database', 'reactome')
wikipathways_pathme_universe_graph_no_flatten = get_subgraph_by_annotation_value(pathme_universe_graph_no_flatten, 'database', 'wikipathways')


##### KEGG

In [36]:
count_functions(kegg_pathme_graph_no_flatten)

Counter({'Composite': 1596,
         'Complex': 405,
         'Gene': 6159,
         'Abundance': 1753,
         'Reaction': 3137,
         'BiologicalProcess': 112})

##### Reactome

In [37]:
count_functions(reactome_pathme_universe_graph_no_flatten)

Counter({'Abundance': 2732,
         'Reaction': 9645,
         'Complex': 8798,
         'Gene': 8952,
         'Composite': 9})

##### Wikipathways

In [38]:
count_functions(wikipathways_pathme_universe_graph_no_flatten)

Counter({'Abundance': 815, 'Gene': 2648, 'Reaction': 800})

### Background  Matrix

In [39]:
then = time.time()

In [ ]:
background_mat = regularised_laplacian_kernel(pathme_universe_graph_no_flatten)

/Users/jmarinllao/Documents/CREB/diffuPy/src/diffuPy/miscellaneous.py:19: UserWarning: Graph must be undirected, so it is converted to undirected.
  warnings.warn('Graph must be undirected, so it is converted to undirected.')
Node name nor id not labeled: composite(p(HGNC:AKT1), p(HGNC:AKT2), p(HGNC:AKT3))
Node name nor id not labeled: composite(p(HGNC:MAPK11), p(HGNC:MAPK12), p(HGNC:MAPK13), p(HGNC:MAPK14))
Node name nor id not labeled: composite(p(HGNC:MAP2K4), p(HGNC:MAP2K7))
Node name nor id not labeled: composite(p(HGNC:MAP2K3), p(HGNC:MAP2K6))
Node name nor id not labeled: composite(p(HGNC:PIK3CA), p(HGNC:PIK3CB), p(HGNC:PIK3CD), p(HGNC:PIK3R1), p(HGNC:PIK3R2), p(HGNC:PIK3R3))
Node name nor id not labeled: composite(p(HGNC:PLCG1), p(HGNC:PLCG2))
Node name nor id not labeled: composite(p(HGNC:MAPK1), p(HGNC:MAPK3))
Node name nor id not labeled: composite(p(HGNC:MAP2K1), p(HGNC:MAP2K2))
Node name nor id not labeled: composite(p(HGNC:HRAS), p(HGNC:KRAS), p(HGNC:NRAS))
Node name nor 

Node name nor id not labeled: rxn(reactants(a(CHEBI:"3-hydroxypyruvic acid")), products(a(CHEBI:"D-glyceric acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"(S)-malic acid")), products(a(CHEBI:"oxaloacetic acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"oxaloacetic acid")), products(a(CHEBI:"(S)-malic acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"2-phosphoglycolic acid")), products(a(CHEBI:glycolate)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"acetyl-CoA")), products(a(CHEBI:"acetoacetyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"(S)-methylmalonyl-CoA")), products(a(CHEBI:"(R)-methylmalonyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"propionyl-CoA")), products(a(CHEBI:"(S)-methylmalonyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"(R)-methylmalonyl-CoA")), products(a(CHEBI:"succinyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"glyoxylic acid")), products(a(CHEBI:"3-h

Node name nor id not labeled: composite(p(HGNC:TLR7), p(HGNC:TLR8))
Node name nor id not labeled: composite(p(HGNC:FOS), p(HGNC:JUN))
Node name nor id not labeled: composite(p(HGNC:CCL4), p(HGNC:CCL4L1), p(HGNC:CCL4L1))
Node name nor id not labeled: complex(p(HGNC:MYD88), p(HGNC:TIRAP), p(HGNC:TOLLIP))
Node name nor id not labeled: complex(p(HGNC:IRAK1), p(HGNC:IRAK4))
Node name nor id not labeled: complex(p(HGNC:MYD88), p(HGNC:TIRAP))
Node name nor id not labeled: complex(p(HGNC:IKBKE), p(HGNC:TBK1))
Node name nor id not labeled: complex(p(HGNC:RIPK1), p(HGNC:TRAF6))
Node name nor id not labeled: complex(p(HGNC:CHUK), p(HGNC:IKBKB), p(HGNC:IKBKG))
Node name nor id not labeled: composite(p(HGNC:TLN1), p(HGNC:TLN2))
Node name nor id not labeled: composite(p(HGNC:ITGAL), p(HGNC:ITGB2))
Node name nor id not labeled: composite(p(HGNC:IL2RB), p(HGNC:IL2RG))
Node name nor id not labeled: composite(p(HGNC:JAK1), p(HGNC:JAK3))
Node name nor id not labeled: composite(p(HGNC:STAT5A), p(HGNC:STAT

Node name nor id not labeled: rxn(reactants(a(KEGG:"gl:G13094")), products(a(KEGG:"gl:G13095")))
Node name nor id not labeled: rxn(reactants(a(KEGG:"gl:G13097")), products(a(KEGG:"gl:G13098")))
Node name nor id not labeled: composite(p(HGNC:ADRB1), p(HGNC:ADRB2), p(HGNC:ADRB3))
Node name nor id not labeled: composite(p(HGNC:PRKG1), p(HGNC:PRKG2))
Node name nor id not labeled: composite(p(HGNC:GNAI1), p(HGNC:GNAI2), p(HGNC:GNAI3))
Node name nor id not labeled: composite(p(HGNC:IRS1), p(HGNC:IRS2), p(HGNC:IRS4))
Node name nor id not labeled: composite(p(HGNC:PTGS1), p(HGNC:PTGS2))
Node name nor id not labeled: composite(p(HGNC:CGA), p(HGNC:TSHB))
Node name nor id not labeled: composite(p(HGNC:"MT-ND1"), p(HGNC:"MT-ND2"), p(HGNC:"MT-ND3"), p(HGNC:"MT-ND4"), p(HGNC:"MT-ND4L"), p(HGNC:"MT-ND5"), p(HGNC:"MT-ND6"), p(HGNC:"NDUFC2-KCTD14"), p(HGNC:NDUFA1), p(HGNC:NDUFA10), p(HGNC:NDUFA11), p(HGNC:NDUFA12), p(HGNC:NDUFA13), p(HGNC:NDUFA2), p(HGNC:NDUFA3), p(HGNC:NDUFA4), p(HGNC:NDUFA4L2), p(HGN

Node name nor id not labeled: rxn(reactants(a(CHEBI:"decanoyl-CoA")), products(a(CHEBI:"3-oxolauroyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"acetyl-CoA")), products(a(CHEBI:"3-oxolauroyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"coenzyme A")), products(a(CHEBI:"decanoyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"3-oxolauroyl-CoA")), products(a(CHEBI:"decanoyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"3-oxolauroyl-CoA")), products(a(CHEBI:"acetyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"(S)-3-hydroxylauroyl-CoA")), products(a(CHEBI:"3-oxolauroyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"3-oxolauroyl-CoA")), products(a(CHEBI:"(S)-3-hydroxylauroyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"(S)-3-hydroxylauroyl-CoA")), products(a(CHEBI:"trans-dodec-2-enoyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"trans-dodec-2-enoyl-CoA")), products(a(CHE

...

Node name nor id not labeled: rxn(reactants(a(CHEBI:"Acrylamide (AA)")), products(a(PUBCHEM.COMPOUND:"Glycidamide (GA)")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"Acrylamide (AA)")), products(a(CHEMSPIDER:"AA-Val")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:DG)), products(a(CHEBI:MG)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:MG)), products(a(CHEBI:Glycerol)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:TG)), products(a(CHEBI:DG)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"25(OH)D3")), products(a(HMDB:"1,25(OH)2D3")))
Node name nor id not labeled: rxn(reactants(), products(a(HMDB:"Previtamin D3")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"Vitamin D3")), products(a(CHEBI:"25(OH)D3")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:ATRA)), products())
Node name nor id not labeled: rxn(reactants(a(CHEBI:"retinol (vit A)")), products(a(CHEBI:retinal)))
Node name nor id not labeled: rxn(reactants(), products(a(CHEBI:ATRA

Node name nor id not labeled: rxn(reactants(), products(a(CHEBI:serine)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"alpha-ketoglutarate")), products(a(CHEBI:succinate)))
Node name nor id not labeled: rxn(reactants(), products(a(CHEBI:DHAP)))
Node name nor id not labeled: rxn(reactants(g(HGNC:PPARGC1A)), products(g(HGNC:PPARGC1A)))
Node name nor id not labeled: rxn(reactants(g(HGNC:HDAC1)), products(g(HGNC:HDAC1)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:Progesterone)), products(a(CHEBI:Deoxycorticosterone)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"17-hydroxypregnenolone")), products(a(CHEBI:DHEA)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"17-hydroxyprogesterone")), products(a(CHEBI:"11-Deoxycortisol")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"17-hydroxypregnenolone")), products(a(CHEBI:"17-hydroxyprogesterone")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:Cholesterol)), products(a(CHEBI:Pregnenolone)))
Node name nor 

Node name nor id not labeled: rxn(reactants(a(CHEBI:"1-alkyl-2-acyl-sn-glycero- 3-phosphoethanolamine")), products())
Node name nor id not labeled: rxn(reactants(), products(a(PUBCHEM.COMPOUND:"NAD+ (Coenzyme I)")))
Node name nor id not labeled: rxn(reactants(), products(a(CHEBI:"1-Alkyl-2-acetyl-sn-glycerol 3-phosphate")))
Node name nor id not labeled: rxn(reactants(), products(a(PUBCHEM.COMPOUND:"1-Alkyldihydroxyacetone phosphate (Octanoyl DHAP)")))
Node name nor id not labeled: rxn(reactants(), products(a(CHEBI:"Phosphatidylinositol-4-phosphate (PIP)")))
Node name nor id not labeled: rxn(reactants(), products(a(CHEBI:"Cardiolipin (DPG, CL)")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"1-Alkyl-2-acetyl-sn-glycerol 3-phosphate")), products())
Node name nor id not labeled: rxn(reactants(a(LIPIDMAPS:"Acyl coenzyme A (Acyl-CoA)")), products())
Node name nor id not labeled: rxn(reactants(a(CHEBI:"1-alkyl-sn-glycero-3-phosphocholine (Lyso PAF)")), products(a(CHEBI:"1-alkyl-2-ace

Node name nor id not labeled: rxn(reactants(a(CHEBI:"N-Carbamyl-beta-aminoisobutyric acid")), products(a(CHEBI:"(S)-beta-aminoisobutyrate")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:glutamine)), products(a(CHEBI:"Carbamoyl-phosphate")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:uracil)), products(a(CHEBI:Dihydrouracil)))
Node name nor id not labeled: rxn(reactants(), products(a(CHEBI:"N-Carbamyl-beta-aminoisobutyric acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:UMP)), products(a(CHEBI:UDP)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"2-Deoxyuridine")), products(a(CHEBI:uracil)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:OMP)), products(a(CHEBI:orotidine)))
Node name nor id not labeled: rxn(reactants(), products(a(CHEBI:cytidine)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:CDP)), products())
Node name nor id not labeled: rxn(reactants(a(CHEBI:Carbamoylaspartate)), products())
Node name nor id not labeled: rxn(reactants(a

Node name nor id not labeled: rxn(reactants(a(CHEBI:"L-tryptophan")), products(a(CHEBI:"5-Hydroxytryptophan")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:Norepinephrine)), products())
Node name nor id not labeled: rxn(reactants(a(CHEBI:"Dihydroxyphenylacetic acid")), products(a(CHEBI:"Homovanillic acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"3-O-methyldopa")), products())
Node name nor id not labeled: rxn(reactants(a(CHEBI:"L-Dopa")), products(a(CHEBI:"3-O-methyldopa")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:Dopamine)), products(a(CHEBI:"3-Methoxytyramine")))
Node name nor id not labeled: rxn(reactants(g(HGNC:DLG4)), products(g(HGNC:DLG4)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"Acetyl-CoA")), products(a(CHEBI:"Acetoacetyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"HMG-CoA")), products())
Node name nor id not labeled: rxn(reactants(), products(a(CHEBI:"(R)-5-phosphomevalonate")))
Node name nor id not labeled: rxn(

In [ ]:
now = time.time()
print("It took: ", now-then, " seconds")

### Row labels

In [ ]:
background_labels = background_mat.rows_labels

In [ ]:
mirnas = dataset1_labels_by_omics['micrornas']

## 3. Dataset label mapping to PathMeEntities

### General mapping

In [ ]:
all_labels_mapping = get_mapping(dataset1_all_labels, background_labels, title = 'Global mapping: ', mirnas=mirnas, print_percentage = True)


### Mapping by subsets

#### Mapping by entity type/omic

In [ ]:
mapping_by_entity, _, _ = get_mapping_subsets(dataset1_labels_by_omics, background_labels, 'entity type/omic', mirnas=mirnas)


In [ ]:
micrornas_mapping = mapping_by_entity['micrornas'][0]
micrornas_mapping

#### Mapping by database

In [ ]:
background_entites_by_db = {db : set(itertools.chain.from_iterable(entities.values())) for db, entities in bg_labels_from_pathmeuniverse_by_db_and_omic.items()}
mapping_by_db, _, _ = get_mapping_subsets(background_entites_by_db, dataset1_all_labels, 'database', mirnas=mirnas, submapping = background_labels, mirnas_mapping = micrornas_mapping, percentage_reference_labels = True)


#### Mapping by entity type and database

In [ ]:
mapping_by_database_and_entity, total_percentage, total_dimention = get_mapping_two_dim_subsets(bg_labels_from_pathmeuniverse_by_db_and_omic, dataset1_all_labels, relative_statistics = dataset1_labels_by_omics, mirnas_mapping = micrornas_mapping)


### Mapping descriptive view

In [ ]:
# TODO

all_count = []
all_percentage = []

entity_type_map = {'metabolite_nodes': 'metabolite', 'mirna_nodes': 'micrornas', 'gene_nodes': 'genes',
                    'bp_nodes': 'bps'}

for db_name, entities_by_type in mapping_by_database_and_entity.items():
    db_count = []
    db_percentage = []

    for entity_type, entities_tupple in entities_by_type[0].items():
        db_count.append(len(entities_tupple[0]))
        db_percentage.append(entities_tupple[1])
        
        
    all_count.append(db_count)
    all_percentage.append(db_percentage)

db_count = []
db_percentage = []
for entity, entities_tupple in mapping_by_entity.items():
    db_count.append(len(entities_tupple[0]))
    db_percentage.append(entities_tupple[1])
    


In [ ]:
import numpy as np

databases = list(mapping_by_database_and_entity.keys())
databases.append('arreu')


entity_types = mapping_by_database_and_entity['kegg'][0].keys()

entity_number = np.array(all_percentage)
entity_count = np.array(all_count)

In [ ]:
databases = mapping_by_database_and_entity.keys()
entity_types = mapping_by_database_and_entity['kegg'][0].keys()

entity_number = np.array(all_percentage)
entity_count = np.array(all_count)

In [ ]:
show_heatmap(entity_number, entity_count, databases, entity_types)

## 4. Score Diffusion with diffuPy: Dataset as input + PathMe as background graph

### Input elements: Dataset input + Background matrix

#### Input vector/matrix construction

In [ ]:
input_mats_by_entity_type = {entity_type: generate_categoric_input_from_labels(all_labels_mapping, ['Dataset 1'], background_mat) for entity_type, map_value in mapping_by_entity.items()}
input_mat = generate_categoric_input_from_labels(all_labels_mapping, ['Dataset 1'], background_mat)


In [ ]:
input_mat_raw = copy.copy(input_mat)
background_mat_raw = copy.copy(background_mat)

input_mat_z = copy.copy(input_mat)
background_mat_z = copy.copy(background_mat)

### Compute diffusion scores

#### Raw scores

In [ ]:
then = time.time()

In [ ]:
raw_scores = diffuse(input_mat_raw, 'ml', K = background_mat_raw)

In [ ]:
now = time.time()
print("It took: ", now-then, " seconds")

#### Normalized z-scores

In [ ]:
then = time.time()

In [ ]:
z_scores = diffuse(input_mat_z, 'ml', K = background_mat_z, z = True)

In [ ]:
now = time.time()
print("It took: ", now-then, " seconds")

## 5. Validation metrics